In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -rf /content/drive/MyDrive/colab/didimdol/* .
!unzip  /content/dataset/dataset.zip -d ./dataset

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델 로딩 및 binary classification 세팅

In [4]:
model = resnet50.ResNet50(weights=None, classes = 1, classifier_activation='sigmoid')
# model = keras.Sequential(model)
# model.add(keras.layers.Dense(1,activation='sigmoid'))
# layers = model.layers
# new_fc = keras.layers.Dense(1, activation = 'sigmoid')(layers[-2].output)
# model = keras.models.Model(model.input, new_fc)

In [5]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# 학습데이터 셋업

In [6]:
trn_data_dir = 'dataset/archive/casting_data/casting_data/train'
val_data_dir = 'dataset/archive/casting_data/casting_data/test'
batch_size = 128
img_height = 224
img_width = 224

In [7]:
train_ds = preprocessing.image_dataset_from_directory(
    trn_data_dir,
    label_mode = 'binary',
    validation_split=0.2,
    subset="training",
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = preprocessing.image_dataset_from_directory(
    trn_data_dir,
    label_mode = 'binary',
    validation_split=0.2,
    subset="validation",
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    #validation_split=0.2,
    label_mode = 'binary',
    #subset="validation",
    shuffle=False,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 6633 files belonging to 2 classes.
Using 5307 files for training.
Found 6633 files belonging to 2 classes.
Using 1326 files for validation.
Found 715 files belonging to 2 classes.


In [10]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    # keras.layers.experimental.preprocessing.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225]),
    # keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
    #                                              input_shape=(img_height, 
    #                                                           img_width,
    #                                                           3)),
    # keras.layers.experimental.preprocessing.RandomRotation(0.1),
    # keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

rescaling1 = keras.layers.experimental.preprocessing.Rescaling(1./255)

rescaling2 = keras.layers.experimental.preprocessing.Rescaling(1./255)

normalized_trn_ds = train_ds.map(lambda x, y: (rescaling1(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (rescaling2(x), y))

In [13]:
train_ds.class_names

['def_front', 'ok_front']

# 모델 학습 셋업

In [14]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [15]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# 모델 학습

In [16]:
epochs=40
history = model.fit(
  normalized_trn_ds,
  validation_data=normalized_val_ds,
  epochs=epochs,
  callbacks=callbacks_list
)

Epoch 1/40
42/42 [==============================] - ETA: 0s - loss: 0.5908 - binary_accuracy: 0.8287
Epoch 1: val_loss improved from inf to 1.77139, saving model to best_model.h5
42/42 [==============================] - 41s 765ms/step - loss: 0.5908 - binary_accuracy: 0.8287 - val_loss: 1.7714 - val_binary_accuracy: 0.4276
Epoch 2/40
42/42 [==============================] - ETA: 0s - loss: 0.0658 - binary_accuracy: 0.9772
Epoch 2: val_loss did not improve from 1.77139
42/42 [==============================] - 30s 682ms/step - loss: 0.0658 - binary_accuracy: 0.9772 - val_loss: 2.4038 - val_binary_accuracy: 0.5724
Epoch 3/40
42/42 [==============================] - ETA: 0s - loss: 0.0496 - binary_accuracy: 0.9844
Epoch 3: val_loss did not improve from 1.77139
42/42 [==============================] - 29s 678ms/step - loss: 0.0496 - binary_accuracy: 0.9844 - val_loss: 2.0507 - val_binary_accuracy: 0.5724
Epoch 4/40
42/42 [==============================] - ETA: 0s - loss: 0.0325 - binary_acc

In [13]:
model.save('./model.h5')

In [14]:
re_model = keras.models.load_model('./model.h5')

In [19]:
output = re_model.predict(x)

In [ ]:
output[output >= 0.5] = 1
output[output < 0.5] = 0